In [ ]:
! [ -d "MaskWearing" ] && echo "skipping" || (wget -nc --no-check-certificate -O mask_wearing_v4.zip "https://public.roboflow.com/ds/gWZZEe9H1W?key=GsYEoVOTBs" && unzip mask_wearing_v4 -d "MaskWearing")
!rm -f "MaskWearing/train/IMG_5690 2_jpg.rf.7198a97a65ae7dde21284ca6e356f7a8.jpg" # Remove file with broken annotation
!rm -f "MaskWearing/train/IMG_5690 2_jpg.rf.7198a97a65ae7dde21284ca6e356f7a8.xml" # Remove file with broken annotation

In [ ]:
import pathlib, json
import os

dataset_folder = pathlib.Path('MaskWearing')

In [ ]:
from sap_computer_vision.datasets import pascal_voc_style as pvs

In [ ]:
d, c = pvs.register('dataset_train',
                    img_dir=dataset_folder / 'train',
                    xml_dir=dataset_folder / 'train')
print(f'{len(d)} examples in the train set!')
d, c = pvs.register('dataset_validation',
                    img_dir=dataset_folder / 'valid',
                    xml_dir=dataset_folder / 'valid',
                    class_names=c)
print(f'{len(d)} examples in the validation set!')
d, c = pvs.register('dataset_test',
                    img_dir=dataset_folder / 'test',
                    xml_dir=dataset_folder / 'test',
                    class_names=c)
print(f'{len(d)} examples in the test set!')

In [ ]:

out_dir = 'object_detecion_model'
if pathlib.Path(out_dir).exists():
    raise RuntimeError('Result folder already exists. Please delete the folder or change the name of the output')

setup_loggers(out_dir)

In [ ]:
import shutil

from sap_computer_vision import setup_loggers, get_cfg, get_config_file
import numpy as np
cfg = get_cfg()

In [ ]:

cfg.merge_from_file(get_config_file('Base-EarlyStopping'))
cfg.merge_from_file(get_config_file('Base-Evaluation'))
cfg.merge_from_file(get_config_file('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'))

In [ ]:
print(cfg)

In [ ]:
import shutil

from sap_computer_vision import setup_loggers, get_cfg, get_config_file
import numpy as np

out_dir = 'object_detecion_model'
if pathlib.Path(out_dir).exists():
    raise RuntimeError('Result folder already exists. Please delete the folder or change the name of the output')

setup_loggers(out_dir)
cfg = get_cfg()
cfg.merge_from_file(get_config_file('Base-EarlyStopping'))
cfg.merge_from_file(get_config_file('Base-Evaluation'))
cfg.merge_from_file(get_config_file('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'))


cfg.SOLVER.MAX_ITER = 100
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.GAMMA = float(np.sqrt(0.1))
cfg.SOLVER.EARLY_STOPPING.ENABLED = False

cfg.SOLVER.WARMUP_ITERS = max(int(0.01 * cfg.SOLVER.MAX_ITER), 50)
cfg.SOLVER.STEPS = [cfg.SOLVER.MAX_ITER * p for p in (0.05, 0.25, 0.375, 0.5, 0.75, 0.9)]
for aug in ['RANDOM_LIGHTING', 'RANDOM_BRIGHTNESS', 'RANDOM_SATURATION', 'RANDOM_CONTRAST', 'CROP']:
    if cfg.get(aug, None) is not None:
        cfg.INPUT[aug].ENABLED = True

cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(c)
        

cfg.OUTPUT_DIR = out_dir
cfg.DATASETS.TRAIN = ('dataset_train', )
cfg.DATASETS.TEST = ('dataset_validation', )

# Adjust to hardware
cfg.SOLVER.IMS_PER_BATCH = 8 
cfg.SOLVER.IMS_PER_BATCH_EVAL = 12
cfg.DATALOADER.NUM_WORKERS = 10

cfg.TEST.EVAL_PERIOD = 250

In [ ]:
cfg.SOLVER.EARLY_STOPPING.ENABLED = False

In [ ]:
out_dir = pathlib.Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
with (out_dir / 'used_config.yaml').open('w') as stream:
    stream.write(cfg.dump())

In [ ]:
from sap_computer_vision.engine import ObjectDetectionTrainer

In [ ]:
trainer = ObjectDetectionTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.DATASETS.TEST = ('dataset_test', )
metrics = trainer.test(cfg, trainer.model)

In [ ]:
from detectron2.data import MetadataCatalog

dl = trainer.build_test_loader(cfg, cfg.DATASETS.TEST)
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

In [ ]:
import torch

from detectron2.structures.instances import Instances
from detectron2.structures.boxes import pairwise_iou

def apply_score_and_iou_cut(inst, max_iou=0.4, score_threshold=0.8):
    if score_threshold is not None:
        mask = inst.scores > score_threshold
        new_fields = {}
        for f in inst.get_fields():
            new_fields[f] = inst.get(f)[mask]
        inst = Instances(inst.image_size, **new_fields)
    if max_iou is not None:
        iou = pairwise_iou(inst.pred_boxes, inst.pred_boxes)
        mask_iou = torch.triu(iou > 0.4, diagonal=1)
        mask = torch.sum(mask_iou, axis=0) == 0 
        new_fields = {}
        for f in inst.get_fields():
            new_fields[f] = inst.get(f)[mask]
        inst = Instances(new_inst.image_size, **new_fields)
    return inst


results = []
for batch in dl:
    trainer.model.eval()
    with torch.no_grad():
        results_batch = trainer.model(batch)
        for i, result_i in enumerate(results_batch):
            input_i = batch[i]
            del input_i['image']
            instances = result_i['instances']
            result_i['instances'] = apply_score_and_iou_cut(instances.to("cpu"))
            results_batch[i] = {**result_i, **input_i}
        results.extend(results_batch)

In [ ]:
from detectron2.data.detection_utils import read_image
from detectron2.utils.visualizer import Visualizer

resuld_idx = np.random.choice([*range(len(results))])
vis = Visualizer(img_rgb=read_image(results[resuld_idx]['file_name']), metadata=metadata)
vis.draw_instance_predictions(results[resuld_idx]['instances']).fig

In [ ]:
import detectron2

In [ ]:
detectron2.__file__